## Install Libraries

We will install the software libraries necessary for this week's tutorials, and test them to make sure they work.

Install [NumPy](https://en.wikipedia.org/wiki/NumPy) (Python support for large, multi-dimensional arrays and matrices ): ```conda install -c conda-forge numpy ```  
Install [MatplotLib](https://matplotlib.org) (Python 2D plotting library ): ```conda install -c conda-forge matplotlib ```  
Install [NumPy Ringbuffer](https://pypi.org/project/numpy_ringbuffer/) (Ringbuffer Data structure library ) ```pip install numpy_ringbuffer```  
Install [pyTorch](https://pytorch.org/get-started/locally/) (Python Machine Learning Library ): ```conda install pytorch torchvision -c pytorch```  
Install [opencv2](https://opencv.org) (Computer Vision library) : ```conda install -c conda-forge  opencv=3.4.1```  
Install [sklearn](https://scikit-learn.org/stable/) (SciKit Machine Learning library): ```conda install scikit-learn```  
Install [librosa](https://librosa.github.io/librosa/) (Music and Audio analysis library ): ```conda install -c conda-forge librosa ```  
Install [PyAudio](https://people.csail.mit.edu/hubert/pyaudio/) (Python bindings for PortAudio, the cross-platform audio I/O library ) : ```conda install -c anaconda pyaudio```  


## Test setup

In the code below, we load ("import") the libraries that we just installed above, to make sure that Python can find the libraries. 


In [1]:
import csv
import os
import wave
import time
import multiprocessing
from threading import Thread
import os
print("Found all built-in libraries.")
import numpy as np
from numpy_ringbuffer import RingBuffer
import matplotlib.pyplot as plt
print("Found all math libraries.")
import pyaudio
import librosa
print("Found all audio libraries.")
import torch
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix
print("Found all machine learning libraries.")
import cv2 as cv
print("Found the computer vision library.")


Found all built-in libraries.
Found all math libraries.
Found all audio libraries.
Found all machine learning libraries.


ImportError: dlopen(/Users/wendyju/anaconda3/lib/python3.7/site-packages/cv2.cpython-37m-darwin.so, 2): Library not loaded: @rpath/libx264.138.dylib
  Referenced from: /Users/wendyju/anaconda3/lib/libavcodec.57.dylib
  Reason: image not found

If the above code runs without a problem, all libraries have installed and can be loaded. While it does not gurantee that everything will work, this is a good sign.


## Create AudioData folder 

In the dashboard of this notebook, go to ``File->Open`` to see the folder this notebook is kept in. 

Create folder by going to ``New->Folder``. (The New button is to the upper right of the file listing.)

Rename the folder by clicking the checkbox before the new ``Untitled Folder`` that appears, and clicking on the Rename button (to the upper left of the file listing).

Name the new folder ``AudioData``.

If you check your workshop folder with Windows Explorer, Finder or the equivalent terminal program, you will see that ``AudioData`` folder.



## Download datasets

We will be using two audio datasets in the first lab: Cats vs. Dogs and UrbanSounds. To download these datasets, you will need to create accounts on [Kaggle](www.kaggle.com), a data science community site run by Google, which has, among other things, datasets for machine learning, and the Urbansounds dataset website.

[Cats vs. Dogs](www.kaggle.com/mmoreaux/audio-cats-and-dogs)

[Urban Sounds](https://urbansounddataset.weebly.com/download-urbansound.html)

When these datasets are finished downloading, unzip the files and move them to the ``AudioData/`` folder.


    